In [36]:
import pandas as pd
import hvplot.pandas
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

In [37]:
# Load the Ames Housing dataset
ames_housing = pd.read_csv("FinalAmeri_Housing_fixed.csv")
ames_housing_df = ames_housing.copy()
ames_housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 29 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Lot Area                                  2930 non-null   int64 
 1   Neighborhood                              2930 non-null   object
 2   Building Type                             2930 non-null   object
 3   House Style                               2930 non-null   object
 4   Overall Quality                           2930 non-null   int64 
 5   Overall Condition                         2930 non-null   int64 
 6   Year Built                                2930 non-null   int64 
 7   Year Remodeled/Added                      2930 non-null   int64 
 8   Unfinished Basement (SF)                  2930 non-null   int64 
 9   Central Air                               2930 non-null   object
 10  1st Floor (SF)                            2930 n

In [38]:
# Verify the categories of these columns
print(ames_housing_df[['Neighborhood','Building Type','House Style','Kitchen Quality','Garage Type']].value_counts())

# One-hot encode categorical columns
neighborhood = pd.get_dummies(ames_housing_df['Neighborhood'])
building_type = pd.get_dummies(ames_housing_df['Building Type'])
house_style = pd.get_dummies(ames_housing_df['House Style'])
kitchen_quality = pd.get_dummies(ames_housing_df['Kitchen Quality'])
garage_type = pd.get_dummies(ames_housing_df['Garage Type'])
                              
# Concatenate the ames_housing_df and the categorical_encoded DataFrame
ames_housing_df = pd.concat([ames_housing_df, neighborhood, building_type, house_style, kitchen_quality, garage_type], axis = 1)
ames_housing_df
# Drop the original categorical columns
ames_housing_df = ames_housing_df.drop(columns = ['Neighborhood','Building Type','House Style','Kitchen Quality','Garage Type'])

# Display the DataFrame
ames_housing_df.head()

Neighborhood  Building Type  House Style  Kitchen Quality  Garage Type
NAmes         1Fam           1Story       TA               Attchd         175
CollgCr       1Fam           1Story       Gd               Attchd          92
                             2Story       Gd               Attchd          73
Somerst       1Fam           1Story       Gd               Attchd          58
NAmes         1Fam           1Story       TA               Detchd          55
                                                                         ... 
NPkVill       Twnhs          2Story       Gd               Attchd           1
NAmes         TwnhsE         SFoyer       TA               Attchd           1
Edwards       1Fam           2.5Unf       Gd               Detchd           1
NAmes         Duplex         SLvl         TA               Attchd           1
OldTown       1Fam           1Story       Fa               0                1
Length: 578, dtype: int64


,Lot Area,Overall Quality,Overall Condition,Year Built,Year Remodeled/Added,Unfinished Basement (SF),Central Air,1st Floor (SF),2nd Floor (SF),Above Ground Living Area (SF),...,Gd,Po,TA,0,2Types,Attchd,Basment,BuiltIn,CarPort,Detchd
0,31770,6,5,1960,1960,441,Y,1656,0,1656,...,0,0,1,0,0,1,0,0,0,0
1,11622,5,6,1961,1961,270,Y,896,0,896,...,0,0,1,0,0,1,0,0,0,0
2,14267,6,6,1958,1958,406,Y,1329,0,1329,...,1,0,0,0,0,1,0,0,0,0
3,11160,7,5,1968,1968,1045,Y,2110,0,2110,...,0,0,0,0,0,1,0,0,0,0
4,13830,5,5,1997,1998,137,Y,928,701,1629,...,0,0,1,0,0,1,0,0,0,0


In [39]:
# Encoding the Remodeled Y/N column using a custom function
def encode_remodeled(value):
    """
    This function encodes 'Y' as 1 and 'N' as 0.
    """
    if value == "Y":
        return 1
    else:
        return 0
# Call the encode_remodeled function on the Remodeled Y/N column
ames_housing_df['Remodeled Y/N'] = ames_housing_df['Remodeled Y/N'].apply(encode_remodeled)

# Encoding the Central Air column using a custom function
def encode_central_air(value):
    """
    This function encodes 'Y' as 1 and 'N' as 0.
    """
    if value == "Y":
        return 1
    else:
        return 0
# call the encode_central_air function on the Central Air column
ames_housing_df['Central Air'] = ames_housing_df['Central Air'].apply(encode_central_air)

# Display the DataFrame
ames_housing_df.head()

,Lot Area,Overall Quality,Overall Condition,Year Built,Year Remodeled/Added,Unfinished Basement (SF),Central Air,1st Floor (SF),2nd Floor (SF),Above Ground Living Area (SF),...,Gd,Po,TA,0,2Types,Attchd,Basment,BuiltIn,CarPort,Detchd
0,31770,6,5,1960,1960,441,1,1656,0,1656,...,0,0,1,0,0,1,0,0,0,0
1,11622,5,6,1961,1961,270,1,896,0,896,...,0,0,1,0,0,1,0,0,0,0
2,14267,6,6,1958,1958,406,1,1329,0,1329,...,1,0,0,0,0,1,0,0,0,0
3,11160,7,5,1968,1968,1045,1,2110,0,2110,...,0,0,0,0,0,1,0,0,0,0
4,13830,5,5,1997,1998,137,1,928,701,1629,...,0,0,1,0,0,1,0,0,0,0


In [40]:
# Extract numeric columns
numeric_columns = ["Lot Area", "Overall Quality", "Overall Condition", "Year Built", "Year Remodeled/Added", 
                   "Unfinished Basement (SF)", "1st Floor (SF)", "2nd Floor (SF)", "Above Ground Living Area (SF)", 
                   "Bedrooms", "Total Rooms", "Garage Cars", "Garage Area", "Wood Deck (SF)", "Open Porch (SF)", 
                   "Enclosed Porch", "Screen Porch", "Year Sold", "Total Bathrooms", "Total Finished Basement (SF)", "Finished Basement/Abv Ground Living Area"]
# Scale numeric columns
scaler = StandardScaler()
numeric_encoded = scaler.fit_transform(ames_housing_df[numeric_columns])

print(numeric_encoded)

[[ 2.74438073 -0.06725371 -0.50671843 ... -0.46580405  0.30763128
   0.39912295]
 [ 0.18709726 -0.7760788   0.39309066 ... -1.52873697  0.25105678
  -0.6371134 ]
 [ 0.5228137  -0.06725371  0.39309066 ... -0.46580405  0.90271124
   0.34250521]
 ...
 [ 0.03719892 -0.7760788  -0.50671843 ... -0.46580405 -0.32516501
  -0.90176842]
 [-0.01750572 -0.7760788  -0.50671843 ... -0.46580405  1.47055162
   0.77833015]
 [-0.06611797  0.64157138 -0.50671843 ...  0.59712887  0.55697816
   1.00875119]]


In [41]:
# Create a DataFrame of the scaled numeric data
numeric_encoded_df = pd.DataFrame(numeric_encoded, columns = ["Lot Area", "Overall Quality", "Overall Condition", "Year Built", "Year Remodeled/Added", 
                   "Unfinished Basement (SF)", "1st Floor (SF)", "2nd Floor (SF)", "Above Ground Living Area (SF)", 
                   "Bedrooms", "Total Rooms", "Garage Cars", "Garage Area", "Wood Deck (SF)", "Open Porch (SF)", 
                   "Enclosed Porch", "Screen Porch", "Year Sold", "Total Bathrooms", "Total Finished Basement (SF)", "Finished Basement/Abv Ground Living Area"])

# Replace the original data with the columns of information from the scaled data
ames_housing_df["Lot Area"] = numeric_encoded_df["Lot Area"]
ames_housing_df["Overall Quality"] = numeric_encoded_df["Overall Quality"]
ames_housing_df["Overall Condition"] = numeric_encoded_df["Overall Condition"]
ames_housing_df["Year Built"] = numeric_encoded_df["Year Built"]
ames_housing_df["Year Remodeled/Added"] = numeric_encoded_df["Year Remodeled/Added"]
ames_housing_df["Unfinished Basement (SF)"] = numeric_encoded_df["Unfinished Basement (SF)"]
ames_housing_df["1st Floor (SF)"] = numeric_encoded_df["1st Floor (SF)"]
ames_housing_df["2nd Floor (SF)"] = numeric_encoded_df["2nd Floor (SF)"]
ames_housing_df["Above Ground Living Area (SF)"] = numeric_encoded_df["Above Ground Living Area (SF)"]
ames_housing_df["Bedrooms"] = numeric_encoded_df["Bedrooms"]
ames_housing_df["Total Rooms"] = numeric_encoded_df["Total Rooms"]
ames_housing_df["Garage Cars"] = numeric_encoded_df["Garage Cars"]
ames_housing_df["Garage Area"] = numeric_encoded_df["Garage Area"]
ames_housing_df["Wood Deck (SF)"] = numeric_encoded_df["Wood Deck (SF)"]
ames_housing_df["Open Porch (SF)"] = numeric_encoded_df["Open Porch (SF)"]
ames_housing_df["Enclosed Porch"] = numeric_encoded_df["Enclosed Porch"]
ames_housing_df["Screen Porch"] = numeric_encoded_df["Screen Porch"]
ames_housing_df["Year Sold"] = numeric_encoded_df["Year Sold"]
ames_housing_df["Total Bathrooms"] = numeric_encoded_df["Total Bathrooms"]
ames_housing_df["Total Finished Basement (SF)"] = numeric_encoded_df["Total Finished Basement (SF)"]
ames_housing_df["Finished Basement/Abv Ground Living Area"] = numeric_encoded_df["Finished Basement/Abv Ground Living Area"]

# Display the DataFrame
ames_housing_df

,Lot Area,Overall Quality,Overall Condition,Year Built,Year Remodeled/Added,Unfinished Basement (SF),Central Air,1st Floor (SF),2nd Floor (SF),Above Ground Living Area (SF),...,Gd,Po,TA,0,2Types,Attchd,Basment,BuiltIn,CarPort,Detchd
0,2.744381,-0.067254,-0.506718,-0.375537,-1.163488,-0.268671,1,1.267003,-0.783185,0.309265,...,0,0,1,0,0,1,0,0,0,0
1,0.187097,-0.776079,0.393091,-0.342468,-1.115542,-0.657780,1,-0.672643,-0.783185,-1.194427,...,0,0,1,0,0,1,0,0,0,0
2,0.522814,-0.067254,0.393091,-0.441674,-1.259380,-0.348313,1,0.432445,-0.783185,-0.337718,...,1,0,0,0,0,1,0,0,0,0
3,0.128458,0.641571,-0.506718,-0.110988,-0.779919,1.105726,1,2.425687,-0.783185,1.207523,...,0,0,0,0,0,1,0,0,0,0
4,0.467348,-0.776079,-0.506718,0.848000,0.658466,-0.960420,1,-0.590974,0.853432,0.255844,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,-0.280621,-0.067254,0.393091,0.418109,-0.012780,-0.853472,1,-0.399561,-0.783185,-0.982723,...,0,0,1,0,0,0,0,0,0,1
2926,-0.160296,-0.776079,-0.506718,0.385040,-0.060726,-0.728320,1,-0.657330,-0.783185,-1.182556,...,0,0,1,0,0,1,0,0,0,0
2927,0.037199,-0.776079,-0.506718,0.682657,0.370789,0.036245,1,-0.483783,-0.783185,-1.048015,...,0,0,1,1,0,0,0,0,0,0
2928,-0.017506,-0.776079,-0.506718,0.087423,-0.444296,-0.828442,1,0.585575,-0.783185,-0.219006,...,0,0,1,0,0,1,0,0,0,0


In [42]:
# Create a list to store inertia values and the values of k
inertia = []

# Create a list to set the range of k values to test
k = list(range(1, 11))

for i in k:
    kmeans = KMeans(n_clusters = i, random_state = 42)
    kmeans.fit(ames_housing_df)
    inertia.append(kmeans.inertia_)
    
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

df_elbow.head()

,k,inertia
0,1,1.869254e+13
1,2,7.092253e+12
2,3,3.855123e+12
3,4,2.435386e+12
4,5,1.736593e+12


In [43]:
# Plot the Elbow curve
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [44]:
# Perform K-Means clustering
num_clusters = 3
kmeans = KMeans(n_clusters = num_clusters, random_state = 42)
cluster_labels = kmeans.fit_predict(ames_housing_df)

# Add cluster labels to the DataFrame
ames_housing_df['Cluster'] = cluster_labels

# Logarithmic transformation of Sale Price
ames_housing_df['Log_Sale_Price'] = np.log1p(ames_housing_df['Sale Price'])

abc = ames_housing_df[['Sale Price', 'Total Rooms', 'Cluster']]
abc.to_csv('totalrooms.csv')

In [45]:
# Plot using hvplot.scatter
scatter_plot_1 = ames_housing_df.hvplot.scatter(
    x='Total Rooms',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Total Rooms',
    xlabel='Total Rooms',
    ylabel='Sale Price'
)
    
scatter_plot_1.opts(yformatter="%.0f")


:Scatter   [Total Rooms]   (Sale Price,Cluster)

In [46]:
# Plot using hvplot.scatter
scatter_plot_2 = ames_housing_df.hvplot.scatter(
    x='Year Built',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Year Built',
    xlabel='Year Built',
    ylabel='Sale Price'
)
scatter_plot_2.opts(yformatter="%.0f")

:Scatter   [Year Built]   (Sale Price,Cluster)

In [12]:
# Plot using hvplot.scatter
scatter_plot_3 = ames_housing_df.hvplot.scatter(
    x='Overall Quality',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Overall Quality',
    xlabel='Overall Quality',
    ylabel='Sale Price'
)
scatter_plot_3.opts(yformatter="%.0f")

:Scatter   [Overall Quality]   (Sale Price,Cluster)

In [13]:
# Plot using hvplot.scatter
scatter_plot_4 = ames_housing_df.hvplot.scatter(
    x='Overall Condition',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Overall Condition',
    xlabel='Overall Condition',
    ylabel='Sale Price'
)
scatter_plot_4.opts(yformatter="%.0f")

:Scatter   [Overall Condition]   (Sale Price,Cluster)

In [14]:
# Plot using hvplot.scatter
scatter_plot_5 = ames_housing_df.hvplot.scatter(
    x='Wood Deck (SF)',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Wood Deck (SF)',
    xlabel='Wood Deck (SF)',
    ylabel='Sale Price'
)
scatter_plot_5.opts(yformatter="%.0f")

:Scatter   [Wood Deck (SF)]   (Sale Price,Cluster)

In [15]:
# Plot using hvplot.scatter
scatter_plot_6 = ames_housing_df.hvplot.scatter(
    x='Finished Basement/Abv Ground Living Area',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Basement/Living Area',
    xlabel='Finished Basement/Abv Ground Living Area',
    ylabel='Sale Price'
)
scatter_plot_6.opts(yformatter="%.0f")

:Scatter   [Finished Basement/Abv Ground Living Area]   (Sale Price,Cluster)

In [16]:
# Plot using hvplot.scatter
scatter_plot_7 = ames_housing_df.hvplot.scatter(
    x='Lot Area',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Lot Area',
    xlabel='Lot Area',
    ylabel='Sale Price'
)
scatter_plot_7.opts(yformatter="%.0f")

:Scatter   [Lot Area]   (Sale Price,Cluster)

In [47]:
# Plot using hvplot.scatter
scatter_plot_8 = ames_housing_df.hvplot.scatter(
    x='Garage Cars',
    y='Sale Price',
    c='Cluster',
    cmap='viridis',
    title='Cluster Analysis on Sale Price vs Garage Cars',
    xlabel='Garage Cars',
    ylabel='Sale Price'
)
scatter_plot_8.opts(yformatter="%.0f")


:Scatter   [Garage Cars]   (Sale Price,Cluster)

In [17]:
# Calculate mean sale price for each neighborhood
neighborhood_mean_prices = {}
for neighborhood_col in neighborhood.columns:
    mean_price = ames_housing_df[ames_housing_df[neighborhood_col] == 1]['Sale Price'].mean()
    neighborhood_mean_prices[neighborhood_col] = mean_price

# Covert the dictionary to a DataFrame 
neighborhood_mean_prices_df = pd.DataFrame(list(neighborhood_mean_prices.items()), columns=['Neighborhood', 'Sale Price'])

# Display the DataFrame
neighborhood_mean_prices_df


,Neighborhood,Sale Price
0,Blmngtn,196661.678571
1,Blueste,143590.000000
2,BrDale,105608.333333
3,BrkSide,124756.250000
4,ClearCr,208662.090909
5,CollgCr,201803.434457
6,Crawfor,207550.834951
7,Edwards,130843.381443
8,Gilbert,190646.575758
9,Greens,193531.250000


In [18]:
# Create a scatter plot
neighborhood_meanPrice_plot = neighborhood_mean_prices_df.hvplot.scatter(
    x='Neighborhood',
    y='Sale Price',
    title='Mean Sale Price by Neighborhood',
    xlabel='Neighborhood',
    ylabel='Mean Sale Price',
    rot = 70,
    color='Neighborhood'    
)
neighborhood_meanPrice_plot

:Scatter   [Neighborhood]   (Sale Price)

In [19]:
# Calculate mean sale price for each garage type
garage_type_mean_prices = {}
for garage_type_col in garage_type.columns:
    mean_price_garage = ames_housing_df[ames_housing_df[garage_type_col] == 1]['Sale Price'].mean()
    garage_type_mean_prices[garage_type_col] = mean_price_garage
    
print(garage_type_mean_prices)
    
# Covert the dictionary to a DataFrame 
garage_type_mean_prices_df = pd.DataFrame(list(garage_type_mean_prices.items()), columns=['Garage Type', 'Sale Price'])

# Display the DataFrame
garage_type_mean_prices_df


{'0': 104949.25477707006, '2Types': 154639.13043478262, 'Attchd': 203772.28365106875, 'Basment': 150473.41666666666, 'BuiltIn': 249344.1505376344, 'CarPort': 105566.66666666667, 'Detchd': 132468.47186700767}


,Garage Type,Sale Price
0,0,104949.254777
1,2Types,154639.130435
2,Attchd,203772.283651
3,Basment,150473.416667
4,BuiltIn,249344.150538
5,CarPort,105566.666667
6,Detchd,132468.471867


In [20]:
# Map abbreviations to full names
garage_type_map = {
    '0': 'No Garage',
    '2Types': 'Two Types',
    'Attchd': 'Attached',
    'Basment': 'Basement',
    'BuiltIn': 'Built-in',
    'CarPort': 'Carport',
    'Detchd': 'Detached'
}
# Replace abbrevations to full names
garage_type_mean_prices_df['Garage Type'] = garage_type_mean_prices_df['Garage Type'].replace(garage_type_map)

garage_type_mean_prices_df

,Garage Type,Sale Price
0,No Garage,104949.254777
1,Two Types,154639.130435
2,Attached,203772.283651
3,Basement,150473.416667
4,Built-in,249344.150538
5,Carport,105566.666667
6,Detached,132468.471867


In [48]:
# Create a scatter plot
garage_meanPrice_plot = garage_type_mean_prices_df.hvplot.barh(
    x='Garage Type',
    y='Sale Price',
    title='Mean Sale Price by Garage Type',
    xlabel='Garage Type',
    ylabel='Mean Sale Price'    
)
garage_meanPrice_plot.opts(xformatter="%.0f")

:Bars   [Garage Type]   (Sale Price)

In [22]:
# Calculate mean sale price for each house style
house_style_mean_prices = {}
for house_style_col in house_style.columns:
    mean_price_garage = ames_housing_df[ames_housing_df[house_style_col] == 1]['Sale Price'].mean()
    house_style_mean_prices[house_style_col] = mean_price_garage
    
print(house_style_mean_prices)
    
# Covert the dictionary to a DataFrame 
house_style_mean_prices_df = pd.DataFrame(list(house_style_mean_prices.items()), columns=['House Style', 'Sale Price'])

# Display the DataFrame
house_style_mean_prices_df

{'1.5Fin': 137529.92356687898, '1.5Unf': 109663.15789473684, '1Story': 178699.87508440242, '2.5Fin': 220000.0, '2.5Unf': 177158.33333333334, '2Story': 206990.15922107676, 'SFoyer': 143472.6626506024, 'SLvl': 165527.3828125}


,House Style,Sale Price
0,1.5Fin,137529.923567
1,1.5Unf,109663.157895
2,1Story,178699.875084
3,2.5Fin,220000.000000
4,2.5Unf,177158.333333
5,2Story,206990.159221
6,SFoyer,143472.662651
7,SLvl,165527.382812


In [23]:
# Create a scatter plot
houseStyle_meanPrice_plot = house_style_mean_prices_df.hvplot.bar(
    x='House Style',
    y='Sale Price',
    title='Mean Sale Price by House Style',
    xlabel='House Style',
    ylabel='Mean Sale Price'    
)
houseStyle_meanPrice_plot

:Bars   [House Style]   (Sale Price)

In [24]:
# Calculate mean sale price for each building type
building_type_mean_prices = {}
for building_type_col in building_type.columns:
    mean_price_garage = ames_housing_df[ames_housing_df[building_type_col] == 1]['Sale Price'].mean()
    building_type_mean_prices[building_type_col] = mean_price_garage
    
print(building_type_mean_prices)
    
# Covert the dictionary to a DataFrame 
building_type_mean_prices_df = pd.DataFrame(list(building_type_mean_prices.items()), columns=['Building Type', 'Sale Price'])

# Display the DataFrame
building_type_mean_prices_df

{'1Fam': 184812.04123711342, '2fmCon': 125581.70967741935, 'Duplex': 139808.9357798165, 'Twnhs': 135934.05940594058, 'TwnhsE': 192311.91416309014}


,Building Type,Sale Price
0,1Fam,184812.041237
1,2fmCon,125581.709677
2,Duplex,139808.935780
3,Twnhs,135934.059406
4,TwnhsE,192311.914163


In [25]:
# Create a scatter plot
building_meanPrice_plot = building_type_mean_prices_df.hvplot.barh(
    x='Building Type',
    y='Sale Price',
    title='Mean Sale Price by Building Type',
    xlabel='Building Type',
    ylabel='Mean Sale Price',   
)
building_meanPrice_plot

:Bars   [Building Type]   (Sale Price)

In [26]:
# Calculate mean sale price for each kitchen quality
kitchen_quality_mean_prices = {}
for kitchen_quality_col in kitchen_quality.columns:
    mean_price_garage = ames_housing_df[ames_housing_df[kitchen_quality_col] == 1]['Sale Price'].mean()
    kitchen_quality_mean_prices[kitchen_quality_col] = mean_price_garage
    
print(kitchen_quality_mean_prices)
    
# Covert the dictionary to a DataFrame 
kitchen_quality_mean_prices_df = pd.DataFrame(list(kitchen_quality_mean_prices.items()), columns=['Kitchen Quality', 'Sale Price'])

# Display the DataFrame
kitchen_quality_mean_prices_df

# Map abbreviations to full names
kitchen_quality_map = {'Ex': 'Excellent', 'Fa': 'Fair', 'Gd': 'Good', 'Po': 'Poor', 'TA': 'Typical/Average'}

# Replace the abbreaviations to full names
kitchen_quality_mean_prices_df['Kitchen Quality'] = kitchen_quality_mean_prices_df['Kitchen Quality'].replace(kitchen_quality_map)

kitchen_quality_mean_prices_df

{'Ex': 337339.34146341466, 'Fa': 105907.04285714286, 'Gd': 210835.5827586207, 'Po': 107500.0, 'TA': 139549.94779116465}


,Kitchen Quality,Sale Price
0,Excellent,337339.341463
1,Fair,105907.042857
2,Good,210835.582759
3,Poor,107500.000000
4,Typical/Average,139549.947791


In [27]:
# Create a scatter plot
kitchen_meanPrice_plot = kitchen_quality_mean_prices_df.hvplot.barh(
    x='Kitchen Quality',
    y='Sale Price',
    title='Mean Sale Price by Kitchen Quality',
    xlabel='Kitchen Quality',
    ylabel='Mean Sale Price',   
)
kitchen_meanPrice_plot

:Bars   [Kitchen Quality]   (Sale Price)